In [2]:
from keras.datasets import cifar10
from keras.utils import np_utils
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
(X_train, y_train),(X_test, y_test) = cifar10.load_data()

In [ ]:
print('Training Images: {}'.format(X_train.shape))
print('Testing Images: {}'.format(X_test.shape))

In [ ]:
for i in range(0,9):
    plt.subplot(330 + 1 + i)
    img = X_train[i]
    plt.imshow(img)
    
plt.show()

In [ ]:
seed = 6
np.random.seed(seed)

(X_train, y_train),(X_test, y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test= X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)
num_class = Y_test.shape[1]
print(num_class)
print(Y_train.shape)
print(Y_train[0])

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Activation, Conv2D, GlobalAveragePooling2D
from keras.optimizers import SGD

In [ ]:
def allcnn(weights = None):
    model = Sequential()
    model.add(Conv2D(96, (3,3), padding = 'same', input_shape=(32,32,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(96, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(96, (3,3), padding = 'same',strides= (2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(192, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(192, (3,3), padding = 'same',strides=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(192, (1,1), padding = 'valid'))
    model.add(Activation('relu'))
    model.add(Conv2D(10, (1,1), padding = 'valid'))
    
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))
    
    if weights:
        model.load_weights(weights)
        
    return model

In [ ]:
learning_rate = 0.01
weight_decay = 1e-6
momentum = 0.9

weights = 'all_cnn_weights_0.9088_0.4994.hdf5'
model = allcnn(weights)

sgd = SGD(lr=learning_rate, decay=weight_decay,momentum=momentum,nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

scores = model.evaluate(X_test,Y_test, verbose=1)
print('Accuracy: {}'.format(scores[1]))

In [ ]:
classes = range(0,10)

names = ['airplane',
        'automobile',
        'bird',
        'cat',
        'deer',
        'dog',
        'frog',
        'horse',
        'ship',
        'truck']

class_labels = dict(zip(classes, names))
print(class_labels)

In [ ]:
batch = X_test[100:109]
labels = np.argmax(Y_test[100:109], axis = -1)

predictions = model.predict(batch, verbose = 1)
print(predictions)

In [ ]:
for image in predictions:
    print(np.sum(image))

In [ ]:
class_result = np.argmax(predictions, axis = -1)
print(class_result)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize = (15,6))
fig.subplots_adjust(hspace = 1)
axs = axs.flatten()

for i,img in enumerate(batch):
    for key, value in class_labels.items():
        if class_result[i] == key:
            title = 'Prediction: {} \nActual: {}'.format(class_labels[key], class_labels[labels[i]])
            axs[i].set_title(title)
            axs[i].axes.get_xaxis().set_visible(False)
            axs[i].axes.get_yaxis().set_visible(False)
        
    axs[i].imshow(img)
plt.show()